In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv("diabetes.csv")

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [5]:
X=df.drop(columns=['Outcome'])
y=df['Outcome']

In [6]:
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()
X=scaler.fit_transform(X)

In [7]:
X.shape

(768, 8)

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=2)

In [9]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [10]:
model= Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.compile(optimizer="Adam",loss="binary_crossentropy",metrics=['accuracy'])

In [12]:
model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.3979 - loss: 0.8311 - val_accuracy: 0.3571 - val_loss: 0.8232
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4539 - loss: 0.7723 - val_accuracy: 0.5130 - val_loss: 0.7356
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6253 - loss: 0.6792 - val_accuracy: 0.6169 - val_loss: 0.6771
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6642 - loss: 0.6479 - val_accuracy: 0.6688 - val_loss: 0.6290
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7122 - loss: 0.6100 - val_accuracy: 0.7143 - val_loss: 0.5982
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7560 - loss: 0.5763 - val_accuracy: 0.7273 - val_loss: 0.5697
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7516 - loss: 0.5571 - val_accuracy: 0.7468 - val_loss: 0.5473
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7787 - loss: 0.5317 - val_accuracy: 0.7532 - 

In [13]:
# we are cluleess  on how many layers / no of neurons/which optimizer or loss. to choose,
# we can automate this process for finding the best suitable parameters

In [14]:
# we will use keras tuner to do so.

In [15]:
# we will so steps by steps

In [16]:
#step by steps
#1. How to select appropriate optimizer
#2. no of nodes in layer
#3. how to select no of layer
#4. all in all one model

In [17]:
import kerastuner as kt

/tmp/ipython-input-1654478174.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [33]:
#create a function  and create a tuner object to pass it into the function
#our concern for the first step is to just select the proper optimizer

In [36]:
#create the function
def build_model(hp):    #hp stands for hyper-parameter
  model=Sequential()
  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  #this lets the tuner choose one optimizer from ['adam','sgd','rmsprop','adadelta'] for each trial.
  optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])

  model.compile(optimizer=optimizer,loss="binary_crossentropy",metrics=["accuracy"])

  return model

In [37]:
#create a tuner object
tuner= kt.RandomSearch(build_model,objective='val_accuracy',max_trials=4)
#this creates a Keras Tuner object that will run up to 4 trials,
# each time building and training a model using a different optimizer
# (or other hyperparameter combination) to find the one with the best validation accuracy.

Reloading Tuner from ./untitled_project/tuner0.json


In [38]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))
#That line starts the tuning process — it trains the model for up to 5 epochs per trial,
#testing different hyperparameter choices (like optimizers) and
#evaluating each on the validation data to find the best one.

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.48051947355270386

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 15m 52s


In [39]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [40]:
# we got the best optimizer value

In [41]:
model= tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [42]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [43]:
model.fit(X_train,y_train,batch_size=32,initial_epoch=6,epochs=100,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7898 - loss: 0.4126 - val_accuracy: 0.7857 - val_loss: 0.4677
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8070 - loss: 0.3987 - val_accuracy: 0.7857 - val_loss: 0.4670
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7919 - loss: 0.4281 - val_accuracy: 0.7857 - val_loss: 0.4687
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7903 - loss: 0.4054 - val_accuracy: 0.7857 - val_loss: 0.4677
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8052 - loss: 0.4143 - val_accuracy: 0.7922 - val_loss: 0.4690
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8021 - loss: 0.4056 - val_accuracy: 0.7922 - val_loss: 0.4665
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8017 - loss: 0.4195 - val_accuracy: 0.7922 - val_loss: 0.4671
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8154 - loss: 0.4011 - val_accuracy: 0.79

In [44]:
# so for next step,  we will find how many number of nueron there should be in one layer

In [75]:
def build_model(hp):
  model= Sequential()
  units = hp.Int("units",min_value=8,max_value=128)
  model.add(Dense(units=units,activation="relu",input_dim=8))
  model.add(Dense(1,activation="sigmoid"))
   # we have found the best optiimzier above, so we will  use that optimizer .
  model.compile(optimizer="rmsprop",loss="binary_crossentropy",metrics=["accuracy"])

  return model

In [76]:
tuner= kt.RandomSearch(build_model,objective="val_accuracy",max_trials=5,directory="directory",project_name="kerastuner")

Reloading Tuner from directory/kerastuner/tuner0.json


In [77]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [78]:
tuner.get_best_hyperparameters()[0].values

{'units': 112}

In [79]:
model= tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [80]:
model.fit(X_train,y_train,batch_size=32,initial_epoch=6,epochs=100,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7418 - loss: 0.5305 - val_accuracy: 0.7792 - val_loss: 0.4858
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7836 - loss: 0.4721 - val_accuracy: 0.7662 - val_loss: 0.4776
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7839 - loss: 0.4785 - val_accuracy: 0.7662 - val_loss: 0.4702
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7861 - loss: 0.4663 - val_accuracy: 0.7662 - val_loss: 0.4663
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8075 - loss: 0.4398 - val_accuracy: 0.7662 - val_loss: 0.4673
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7586 - loss: 0.4758 - val_accuracy: 0.7727 - val_loss: 0.4686
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7917 - loss: 0.4461 - val_accuracy: 0.7662 - val_loss: 0.4673
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7993 - loss: 0.4415 - val_accuracy: 0.77

In [69]:
#now lets do how to select number of layer

In [83]:
def build_model(hp):
  model=Sequential()

  model.add(Dense(112,activation="relu",input_dim=8))

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    model.add(Dense(72,activation='relu'))

  model.add(Dense(1,activation="sigmoid"))
  model.compile(optimizer="rmsprop",loss="binary_crossentropy",metrics=["accuracy"])

  return model

In [84]:
tuner= kt.RandomSearch(build_model,objective="val_accuracy",max_trials=5,directory="directory",project_name="kerastuner2")

In [85]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 05s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 18s


In [86]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4}

In [87]:
model= tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [90]:
model.fit(X_train,y_train,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7873 - loss: 0.4571 - val_accuracy: 0.7662 - val_loss: 0.4694
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8153 - loss: 0.4069 - val_accuracy: 0.7857 - val_loss: 0.4765
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8221 - loss: 0.3807 - val_accuracy: 0.7662 - val_loss: 0.4813
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8334 - loss: 0.3775 - val_accuracy: 0.7662 - val_loss: 0.5282
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8218 - loss: 0.3883 - val_accuracy: 0.7597 - val_loss: 0.4975
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8266 - loss: 0.3638 - val_accuracy: 0.7468 - val_loss: 0.5530
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8290 - loss: 0.3472 - val_accuracy: 0.7273 - val_loss: 0.5670
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8464 - loss: 0.3741 - val_accuracy: 0.77

In [ ]:
#number of layrs and nodes

In [110]:
def build_models(hp):
    model = Sequential()

    # Number of layers
    num_layers = hp.Int("num_layers", min_value=1, max_value=10)

    for i in range(num_layers):
        # Units and activation per layer
        units = hp.Int("units_" + str(i), min_value=8, max_value=128, step=8)
        activation = hp.Choice("activation_" + str(i), values=["relu", "tanh", "sigmoid"])

        # First layer needs input_dim
        if i == 0:
            model.add(Dense(units=units, activation=activation, input_dim=8))
        else:
            model.add(Dense(units=units, activation=activation))

    # Output layer
    model.add(Dense(1, activation="sigmoid"))

    # Optimizer
    optimizer = hp.Choice("optimizer", values=["adam", "sgd", "rmsprop"])

    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
    return model


In [115]:
tuner = kt.RandomSearch(
    build_models,
    objective="val_accuracy",
    max_trials=3,
    directory="directory",
    project_name="gg"
)

tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

best_hp = tuner.get_best_hyperparameters()[0]
print(best_hp.values)


Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.7597402334213257

Best val_accuracy So Far: 0.7597402334213257
Total elapsed time: 00h 00m 11s
{'num_layers': 1, 'units_0': 40, 'activation_0': 'sigmoid', 'optimizer': 'rmsprop', 'units_1': 24, 'activation_1': 'sigmoid', 'units_2': 32, 'activation_2': 'sigmoid', 'units_3': 24, 'activation_3': 'sigmoid', 'units_4': 24, 'activation_4': 'relu', 'units_5': 80, 'activation_5': 'sigmoid', 'units_6': 24, 'activation_6': 'tanh', 'units_7': 80, 'activation_7': 'relu', 'units_8': 24, 'activation_8': 'sigmoid', 'units_9': 72, 'activation_9': 'sigmoid'}


In [116]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [117]:
model.fit(X_train,y_train,epochs=200,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7051 - loss: 0.5485 - val_accuracy: 0.7792 - val_loss: 0.5296
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7859 - loss: 0.5302 - val_accuracy: 0.7727 - val_loss: 0.5187
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7357 - loss: 0.5379 - val_accuracy: 0.7468 - val_loss: 0.5165
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7677 - loss: 0.5146 - val_accuracy: 0.7532 - val_loss: 0.5093
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7745 - loss: 0.5109 - val_accuracy: 0.7532 - val_loss: 0.5047
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7606 - loss: 0.5124 - val_accuracy: 0.7532 - val_loss: 0.5034
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7559 - loss: 0.5095 - val_accuracy: 0.7597 - val_loss: 0.4969
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7563 - loss: 0.5061 - val_accuracy: 0.75